In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

### 데이터 전처리 방식 지정

In [2]:
mnist = input_data.read_data_sets("./mnist/data", one_hot = True)
X = tf.placeholder(tf.float32, [None, 28*28]) #MNIST = 28*28
Z = tf.placeholder(tf.float32, [None, 128]) #Noise Dimension = 128

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

### GAN의 생성자(Generator) 선언
Noise z를 받아서 fake data를 만드는 Generator
- Fake data와 1 사이의 loss

In [3]:
G_W1 = tf.Variable(tf.random_normal([128,256], stddev=0.01))
G_W2 = tf.Variable(tf.random_normal([256,28*28], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([256]))
G_b2 = tf.Variable(tf.zeros([28*28]))

def generator(noise_z):
  hidden = tf.nn.relu(tf.matmul(noise_z, G_W1)+G_b1)
  output = tf.nn.sigmoid(tf.matmul(hidden, G_W2)+ G_b2)
  return output

### GAN의 구분자(Discriminator) 선언
Real data와 Fake data를 구분하는 Discriminator

- Real data와 1 사이의 loss
- Fake data와 0 사이의 loss

In [4]:
D_W1 = tf.Variable(tf.random_normal([28*28, 256], stddev=0.01))
D_W2 = tf.Variable(tf.random_normal([256, 1], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([256]))
D_b2 = tf.Variable(tf.zeros([1]))

def discriminator(inputs):
  hidden = tf.nn.relu(tf.matmul(inputs, D_W1)+D_b1)
  output = tf.nn.sigmoid(tf.matmul(hidden, D_W2)+D_b2)
  return output

### 생성, loss, optimization

In [5]:
G = generator(Z)

loss_D = -tf.reduce_mean(tf.log(discriminator(X)) + tf.log(1-discriminator(G)))
loss_G = -tf.reduce_mean(tf.log(discriminator(G)))

train_D = tf.train.AdamOptimizer(learning_rate = 0.0002).minimize(loss_D, var_list=[D_W1, D_b1, D_W2, D_b2])
train_G = tf.train.AdamOptimizer(learning_rate = 0.0002).minimize(loss_G, var_list=[G_W1, G_b1, G_W2, G_b2])


### Training and Testing

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

noise_test = np.random.normal(size=(10,128))
for epoch in range(200):
  for i in range(int(mnist.train.num_examples/100)):
    batch_xs, _ = mnist.train.next_batch(100)
    noise = np.random.normal(size=(100,128))
    
    sess.run(train_D, feed_dict={X: batch_xs, Z:noise})
    sess.run(train_G, feed_dict={Z: noise})
  if epoch == 0 or (epoch+1) % 10 == 0:
    samples = sess.run(G, feed_dict={Z: noise_test})
    
    fig, ax = plt.subplots(1,10,figsize=(10,1)) #saving period
    for i in range(10):
      ax[i].set_axis_off()
      ax[i].imshow(np.reshape(samples[i], (28,28)))
    plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
    plt.close(fig)